In [54]:
import numpy as пр
import pandas as pd

In [55]:
#Reading The Dataset
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='BqUs0KYfxzrLPRBCAnlsY1i-20vTS38c8Ks157IQ1BS7',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'vehicleperformanceanalyzer-donotdelete-pr-ccgjs2talld8s5'
object_key = 'car performance.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

datas = pd.read_csv(body)
datas.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [56]:
# Handling Missing Values

datas.isnull().any()

mpg             False
cylinders       False
displacement    False
horsepower      False
weight          False
acceleration    False
model year      False
origin          False
car name        False
dtype: bool

In [57]:
# Splitting The Dataset Into Dependent And Independent Variable.

x = datas.iloc[:,1:8].values

In [58]:
y = datas.iloc[:,0].values

In [59]:
# Split The Dataset Into Train Set And Test Set

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [60]:
# Normalizing

from sklearn.preprocessing import StandardScaler
sd = StandardScaler()
x_train = sd.fit_transform(x_train)
x_test = sd.fit_transform(x_test)

In [61]:
x_train

array([[ 1.49526939,  1.22961301,  1.24359144, ..., -0.79520768,
        -1.13752513, -0.73301171],
       [-0.85285735, -0.92367663, -1.16092059, ...,  1.24411524,
        -1.41177304,  0.5068698 ],
       [-0.85285735, -0.92367663, -0.68001818, ...,  0.05760009,
         1.05645814,  0.5068698 ],
       ...,
       [-0.85285735, -1.206235  , -1.45480539, ...,  1.42950823,
        -0.86327722,  0.5068698 ],
       [ 0.32120602,  0.56706235, -0.09224857, ..., -0.2390287 ,
        -1.41177304, -0.73301171],
       [-0.85285735, -0.99188037, -0.86703579, ...,  0.31715028,
        -0.31478141,  0.5068698 ]])

In [62]:
# Build The Model With The Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor
d = RandomForestRegressor (n_estimators=30,random_state = 0)
d.fit(x_train,y_train)

RandomForestRegressor(n_estimators=30, random_state=0)

In [63]:
# prediction

y_pred = d.predict(x_test)
y_pred

array([14.38333333, 24.25666667, 14.21666667, 20.56666667, 18.47333333,
       30.21666667, 34.63333333, 21.15      , 16.30333333, 25.76      ,
       36.60333333, 36.27      , 19.53666667, 27.32333333, 16.54333333,
       32.99333333, 28.32333333, 27.49666667, 17.03      , 35.82      ,
       16.47333333, 23.54      , 23.16666667, 20.7       , 33.69666667,
       26.45      , 33.79666667, 30.37333333, 31.93666667, 16.57333333,
       20.26666667, 32.99      , 19.79666667, 34.08333333, 20.85666667,
       25.02      , 19.65333333, 17.14      , 34.78333333, 12.76666667,
       13.73333333, 15.2       , 28.32      , 32.76666667, 28.74333333,
       22.68666667, 20.54333333, 16.50666667, 23.38      , 29.88333333,
       34.31666667, 26.5       , 17.63      , 27.78333333, 15.96666667,
       12.96666667, 18.86666667, 26.91666667, 31.95666667, 15.68      ,
       20.81      , 25.97      , 19.84666667, 21.6       , 13.46666667,
       15.33333333, 14.2       , 18.90333333, 24.72666667, 14.21

In [64]:
!pip install ibm_watson_machine_learning

In [65]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey": "APGWCp5rv2AMp9x_hadO6NK6pzJcZAOadcl0zoTjFHyG"
                  }
client = APIClient(wml_credentials)

In [66]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name )['metadata']['id'])

In [67]:
space_uid = guid_from_space_name(client, 'models')
print("Space UID = " + space_uid)

Space UID = 8e9ce932-c021-4332-a82c-9994699d398a


In [68]:
client.set.default_space(space_uid)

'SUCCESS'

In [69]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [73]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [76]:
model_details = client.repository.store_model(model=d,meta_props={
    client.repository.ModelMetaNames.NAME:"churn deployment",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id = client.repository.get_model_id(model_details)

In [77]:
model_id

'1ba6b7e9-4cef-4995-81ed-392f4395ce63'

In [78]:
x_train[0]

array([ 1.49526939,  1.22961301,  1.24359144,  1.38832919, -0.79520768,
       -1.13752513, -0.73301171])

In [79]:
d.predict([[ 1.49526939,  1.22961301,  1.24359144,  1.38832919, -0.79520768,
       -1.13752513, -0.73301171]])

array([14.56666667])

In [ ]:
# Model Evaluation

from sklearn.metrics import r2_score
accuracy = r2_score(y_test,y_pred)
accuracy

In [ ]:
# save the model

import pickle
pickle.dump(d,open('regression.pkl','wb'))